# RAG with Docling and 🦙 LlamaIndex

In [1]:
# requirements for this example:
%pip install -qq docling docling-core python-dotenv llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-vector-stores-milvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tempfile import TemporaryDirectory

from dotenv import load_dotenv
from pydantic import TypeAdapter
from rich.pretty import pprint

load_dotenv()

True

In [3]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Setup

### Helpers

Below we define:

- `DoclingPDFReader` which will be used to create LlamaIndex documents, and
- `HierarchicalJSONNodeParser`, which can be used to create LlamaIndex nodes out of JSON-based documents


In [4]:
from enum import Enum
from pathlib import Path
from typing import Any, Iterable

from llama_index.core.readers.base import BasePydanticReader
from llama_index.core.schema import Document as LIDocument

from docling.document_converter import DocumentConverter

class DocMetaKeys(str, Enum):
    DL_DOC_HASH = "dl_doc_hash"
    ORIGIN = "origin"

class DoclingPDFReader(BasePydanticReader):
    class ParseType(str, Enum):
        MARKDOWN = "markdown"
        JSON = "json"

    parse_type: ParseType = ParseType.MARKDOWN

    def lazy_load_data(
        self,
        file_path: str | Path | Iterable[str] | Iterable[Path],
        *args: Any,
        **load_kwargs: Any,
    ) -> Iterable[LIDocument]:
        file_paths = (
            file_path
            if isinstance(file_path, Iterable) and not isinstance(file_path, str)
            else [file_path]
        )
        converter = DocumentConverter()
        for source in file_paths:
            dl_doc = converter.convert_single(source).output
            match self.parse_type:
                case self.ParseType.MARKDOWN:
                    text = dl_doc.export_to_markdown()
                case self.ParseType.JSON:
                    text = dl_doc.model_dump_json()
                case _:
                    raise RuntimeError(
                        f"Unexpected export type encountered: {self.export_type}"
                    )
            origin = str(source) if isinstance(source, Path) else source
            li_doc = LIDocument(text=text)
            li_doc.metadata = {
                DocMetaKeys.DL_DOC_HASH: dl_doc.file_info.document_hash,
                DocMetaKeys.ORIGIN: origin,
            }
            yield li_doc

In [5]:
from typing import Any, Iterable, Sequence

from docling_core.transforms.chunker import ChunkWithMetadata, HierarchicalChunker
from docling_core.types import Document as DLDocument
from llama_index.core import Document as LIDocument
from llama_index.core.node_parser.interface import NodeParser
from llama_index.core.schema import (
    BaseNode,
    NodeRelationship,
    RelatedNodeType,
    TextNode,
)
from llama_index.core.utils import get_tqdm_iterable


class NodeMetaKeys(str, Enum):
    PATH = "path"
    PAGE = "page"
    BBOX = "bbox"
    ORIGIN = "origin"


class HierarchicalJSONNodeParser(NodeParser):

    def _parse_nodes(
        self,
        nodes: Sequence[BaseNode],
        show_progress: bool = False,
        **kwargs: Any,
    ) -> list[BaseNode]:
        nodes_with_progress: Iterable[BaseNode] = get_tqdm_iterable(
            items=nodes, show_progress=show_progress, desc="Parsing nodes"
        )
        all_nodes: list[BaseNode] = []
        chunker = HierarchicalChunker()
        for input_node in nodes_with_progress:
            li_doc = LIDocument.model_validate(input_node)
            dl_doc: DLDocument = DLDocument.model_validate_json(li_doc.get_content())
            chunk_iter = chunker.chunk(dl_doc=dl_doc)
            for chunk in chunk_iter:
                rels: dict[NodeRelationship, RelatedNodeType] = {
                    NodeRelationship.SOURCE: li_doc.as_related_node_info(),
                }
                excl_doc_meta_keys = [d.value for d in DocMetaKeys]
                excl_node_meta_keys = [n.value for n in NodeMetaKeys]
                excl_meta_keys = excl_doc_meta_keys + excl_node_meta_keys
                node = TextNode(
                    text=chunk.text,
                    excluded_embed_metadata_keys=excl_meta_keys,
                    excluded_llm_metadata_keys=excl_meta_keys,
                    relationships=rels,
                )
                node.metadata = {NodeMetaKeys.PATH: chunk.path}
                if isinstance(chunk, ChunkWithMetadata):
                    node.metadata[NodeMetaKeys.PAGE] = chunk.page
                    node.metadata[NodeMetaKeys.BBOX] = chunk.bbox
                all_nodes.append(node)
        return all_nodes

### Reader and node parser

#### Using JSON

To leverage Docling's rich document structure format, we can namely export to JSON and use the HierarchicalJSONNodeParser accordingly:

In [6]:
reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.JSON)
node_parser = HierarchicalJSONNodeParser()

#### Using Markdown

Alternatively, to just use the flat Markdown export instead of the native document format, one can uncomment and use the following:

In [7]:
# from llama_index.core.node_parser import MarkdownNodeParser

# reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.MARKDOWN)
# node_parser = MarkdownNodeParser()

### Transformations

Our transformations currently include the `node_parser`:

In [8]:
transformations = [node_parser]

One can include add more transformations, e.g. further chunking based on text size / overlap, as shown below:

In [9]:
# from llama_index.core.node_parser import TokenTextSplitter

# splitter = TokenTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=0,
# )
# transformations.append(splitter)

### Embed model

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-small")

### Vector store

In [11]:
INGEST = True  # whether to ingest from scratch or reuse an existing vector store

In [12]:
from llama_index.vector_stores.milvus import MilvusVectorStore

MILVUS_URL = os.environ.get(
    "MILVUS_URL", f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
)
MILVUS_COLL_NAME = os.environ.get("MILVUS_COLL_NAME", "basic_llamaindex_pipeline")
MILVUS_KWARGS = TypeAdapter(dict).validate_json(os.environ.get("MILVUS_KWARGS", "{}"))

vector_store = MilvusVectorStore(
    uri=MILVUS_URL,
    collection_name=MILVUS_COLL_NAME,
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=INGEST,
    **MILVUS_KWARGS,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from llama_index.core import StorageContext, VectorStoreIndex

if INGEST:
    # in this case we ingest the data into the vector store
    docs = reader.load_data(
        file_path="https://arxiv.org/pdf/2206.01062",  # DocLayNet paper
    )
    pprint(docs, max_length=1, max_string=50, max_depth=4)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents=docs,
        embed_model=embed_model,
        storage_context=storage_context,
        transformations=transformations,
    )
else:
    # in this case we just load the vector store index
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model,
    )

[
│   Document(
│   │   id_='83f7b6f1-33e3-493f-8240-95662a93d4dc',
│   │   embedding=None,
│   │   metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c84663'+14, ... +1},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='{"_name":"","type":"pdf-document","description":{"'+173793,
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

### LLM

In [14]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_API_KEY = os.environ.get("HF_API_KEY")

llm = HuggingFaceInferenceAPI(
    token=HF_API_KEY,
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
)

## RAG

In [15]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("How many pages were annotated by humans?")
pprint(query_res, max_length=5, max_string=250, max_depth=6)

Response(
│   response='80863 pages were annotated by humans.',
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='79ee790b-73d8-4268-90d7-301b5cd5e8f4',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
│   │   │   │   │   'origin': 'https://arxiv.org/pdf/2206.01062',
│   │   │   │   │   'path': '$.main-text[36]',
│   │   │   │   │   'page': 2,
│   │   │   │   │   'bbox': [317.11236572265625, 116.19312286376953, 559.7131958007812, 202.27523803710938]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   │   │   │   node_id='83f7b6f1-33e3-493f-8240-95662a93d4dc',
│   │   │   │   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   │   │   │   metadata={...},
│   │   │   │   │   │   hash='10c71d271e8c332f43b561647f58aae7cbf5c8cdb380d0486c553cc72be5102f'
│   │   │   │   │   ),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   │   │   │   node_id='5509c0ef-2890-4bba-aa0f-82c0c389a621',
│   │   │   │   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   │   │   │   metadata={...},
│   │   │   │   │   │   hash='d2593a3a6590fdbc8c1ce8cdb8c0a30f1305d1dcde2ec42d564cff772e10cba7'
│   │   │   │   │   ),
│   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   │   │   │   node_id='47f51f1f-e92f-4d82-b36e-466fa62f8e34',
│   │   │   │   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   │   │   │   metadata={...},
│   │   │   │   │   │   hash='df1e56242d89ec477ed088de11f8bb175f091ae62926228530ebefd3a2b260b4'
│   │   │   │   │   )
│   │   │   │   },
│   │   │   │   text='3 THE DOCLAYNET DATASET\nDocLayNet contains 80863 PDF pages. Among these, 7059 carry two instances of human annotations, and 1591 carry three. This amounts to 91104 total annotation instances. The annotations provide layout information in the shape of'+296,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=None,
│   │   │   │   end_char_idx=None,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8344892859458923
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='e1585b75-17f1-42b1-882a-f44e6ae4d382',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
│   │   │   │   │   'origin': 'https://arxiv.org/pdf/2206.01062',
│   │   │   │   │   'path': '$.main-text[75]',
│   │   │   │   │   'page': 5,
│   │   │   │   │   'bbox': [53.26631546020508, 86.24749755859375, 295.562255859375, 215.95584106445312]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   │   │   │   node_id='83f7b6f1-33e3-493f-8240-95662a93d4dc',
│   │   │   │   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   │   │   │   metadata={...},
│   │   │   │   │   │   hash='10c71d271e8c332f43b561647f58aae7cbf5c8cdb380d0486c553cc72be5102f'
│   │   │   │   │   ),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   │   │   │   node_id='964511c8-a412-47c4-8a3d-e4bf92edbda4',
│   │   │   │   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   │   │   │   metadata={...},
│   │   │   │   │   │   hash='c753eb0a489b37f18e388e